## Task 1: Setup Jupyter Notebook and Initial Project Variables

In [1]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

1.40.76


In [2]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

[2025-11-19 15:38:28,201] p16132 {credentials.py:1364} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [4]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '613468529489')

In [5]:
inference_profile = "us.amazon.nova-micro-v1:0" 
foundation_model = inference_profile[3:]
foundation_model

'amazon.nova-micro-v1:0'

In [6]:
suffix = f"{region}-{account_id}"
agent_name = "hr-assistant-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_description = "Agent for providing HR assistance to manage vacation time"
agent_instruction = "You are an HR agent, helping employees understand HR policies and manage vacation time"
agent_action_group_name = "VacationsActionGroup"
agent_action_group_description = "Actions for getting the number of available vacations days for an employee and confirm new time off"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

## Task 2: Creating the AWS Lambda Function

In [7]:
# creating employee database to be used by lambda function
import sqlite3
import random
from datetime import date, timedelta

# Connect to the SQLite database (creates a new one if it doesn't exist)
conn = sqlite3.connect('employee_database.db')
c = conn.cursor()

# Create the employees table
c.execute('''CREATE TABLE IF NOT EXISTS employees
                (employee_id INTEGER PRIMARY KEY AUTOINCREMENT, employee_name TEXT, employee_job_title TEXT, employee_start_date TEXT, employee_employment_status TEXT)''')

# Create the vacations table
c.execute('''CREATE TABLE IF NOT EXISTS vacations
                (employee_id INTEGER, year INTEGER, employee_total_vacation_days INTEGER, employee_vacation_days_taken INTEGER, employee_vacation_days_available INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Create the planned_vacations table
c.execute('''CREATE TABLE IF NOT EXISTS planned_vacations
                (employee_id INTEGER, vacation_start_date TEXT, vacation_end_date TEXT, vacation_days_taken INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Generate some random data for 10 employees
employee_names = ['John Doe', 'Jane Smith', 'Bob Johnson', 'Alice Williams', 'Tom Brown', 'Emily Davis', 'Michael Wilson', 'Sarah Taylor', 'David Anderson', 'Jessica Thompson']
job_titles = ['Manager', 'Developer', 'Designer', 'Analyst', 'Accountant', 'Sales Representative']
employment_statuses = ['Active', 'Inactive']

for i in range(10):
    name = employee_names[i]
    job_title = random.choice(job_titles)
    start_date = date(2015 + random.randint(0, 7), random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d')
    employment_status = random.choice(employment_statuses)
    c.execute("INSERT INTO employees (employee_name, employee_job_title, employee_start_date, employee_employment_status) VALUES (?, ?, ?, ?)", (name, job_title, start_date, employment_status))
    employee_id = c.lastrowid

    # Generate vacation data for the current employee
    for year in range(date.today().year, date.today().year - 3, -1):
        total_vacation_days = random.randint(10, 30)
        days_taken = random.randint(0, total_vacation_days)
        days_available = total_vacation_days - days_taken
        c.execute("INSERT INTO vacations (employee_id, year, employee_total_vacation_days, employee_vacation_days_taken, employee_vacation_days_available) VALUES (?, ?, ?, ?, ?)", (employee_id, year, total_vacation_days, days_taken, days_available))

        # Generate some planned vacations for the current employee and year
        num_planned_vacations = random.randint(0, 3)
        for _ in range(num_planned_vacations):
            start_date = date(year, random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d')
            end_date = (date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])) + timedelta(days=random.randint(1, 14))).strftime('%Y-%m-%d')
            days_taken = (date(int(end_date[:4]), int(end_date[5:7]), int(end_date[8:])) - date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])))
            c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, days_taken.days))

# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:
%%writefile lambda_function.py
# import os
# import json
# import shutil
# import sqlite3
# from datetime import datetime


# def get_available_vacations_days(employee_id):
#     conn = sqlite3.connect('/tmp/employee_database.db')
#     c = conn.cursor()

#     if employee_id:
#         c.execute("""
#             SELECT employee_vacation_days_available
#             FROM vacations
#             WHERE employee_id = ?
#             ORDER BY year DESC
#             LIMIT 1
#         """, (employee_id,))

#         available_vacation_days = c.fetchone()

#         if available_vacation_days:
#             available_vacation_days = available_vacation_days[0]
#             conn.close()
#             return available_vacation_days
#         else:
#             conn.close()
#             return f"No vacation data found for employed_id {employee_id}"
#     else:
#         conn.close()
#         raise Exception("No employee id provided")

import os
import json
import shutil
import sqlite3
from datetime import datetime

def lambda_handler(event, context):
    # Copiar base de datos a /tmp/ si no existe
    if not os.path.exists('/tmp/employee_database.db'):
        shutil.copy('employee_database.db', '/tmp/employee_database.db')
    
    # Obtener parámetros del evento
    action = event.get('actionGroup', '')
    function = event.get('function', '')
    parameters = event.get('parameters', [])
    
    # Extraer employee_id de los parámetros
    employee_id = None
    for param in parameters:
        if param.get('name') == 'employee_id':
            employee_id = param.get('value')
            break
    
    try:
        if function == 'get_available_vacations_days':
            result = get_available_vacations_days(employee_id)
            return {
                'response': {
                    'actionGroup': action,
                    'function': function,
                    'functionResponse': {
                        'responseBody': {
                            'TEXT': {
                                'body': str(result)
                            }
                        }
                    }
                }
            }
        else:
            return {
                'response': {
                    'actionGroup': action,
                    'function': function,
                    'functionResponse': {
                        'responseBody': {
                            'TEXT': {
                                'body': 'Function not found'
                            }
                        }
                    }
                }
            }
    except Exception as e:
        return {
            'response': {
                'actionGroup': action,
                'function': function,
                'functionResponse': {
                    'responseBody': {
                        'TEXT': {
                            'body': f'Error: {str(e)}'
                        }
                    }
                }
            }
        }

def get_available_vacations_days(employee_id):
    conn = sqlite3.connect('/tmp/employee_database.db')
    c = conn.cursor()

    if employee_id:
        c.execute("""
            SELECT employee_vacation_days_available
            FROM vacations
            WHERE employee_id = ?
            ORDER BY year DESC
            LIMIT 1
        """, (employee_id,))

        available_vacation_days = c.fetchone()

        if available_vacation_days:
            available_vacation_days = available_vacation_days[0]
            conn.close()
            return available_vacation_days
        else:
            conn.close()
            return f"No vacation data found for employee_id {employee_id}"
    else:
        conn.close()
        raise Exception("No employee id provided")


Overwriting lambda_function.py


In [9]:

try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': '9f8d62dc-d652-45a4-98cc-24903ff21b16',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 19 Nov 2025 20:38:52 GMT',
   'x-amzn-requestid': '9f8d62dc-d652-45a4-98cc-24903ff21b16',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [12]:

# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.write("employee_database.db")
z.close()
zip_content = s.getvalue()

# # Create Lambda Function
# lambda_function = lambda_client.create_function(
#     FunctionName=lambda_function_name,
#     Runtime='python3.12',
#     Timeout=180,
#     Role=lambda_iam_role['Role']['Arn'],
#     Code={'ZipFile': zip_content},
#     Handler='lambda_function.lambda_handler'
# )

try:
    # Create Lambda Function
    lambda_function = lambda_client.create_function(
        FunctionName=lambda_function_name,
        Runtime='python3.12',
        Timeout=180,
        Role=lambda_iam_role['Role']['Arn'],
        Code={'ZipFile': zip_content},
        Handler='lambda_function.lambda_handler'
    )
    print("Function created successfully")
except lambda_client.exceptions.ResourceConflictException:
    # Actualizar función existente
    lambda_function = lambda_client.update_function_code(
        FunctionName=lambda_function_name,
        ZipFile=zip_content
    )
    print("Function updated successfully")

Function updated successfully


## Task 3: Create Amazon Bedrock Agent

Time to Create the Agent!

Before you can launch the HR agent into action, you need to give it the right permissions. In this step, you'll:

🔐 Create the agent policies that allow it to invoke Bedrock foundation models.

🛡️ Create the IAM role for the agent and attach the policies.

These steps will allow the agent to safely and successfully use the tools you define.

First, you will create the IAM policy that the agent will need to use the inference profile and invoke the model. 

In the next cell, in the Jupyter notebook, run the following code:

In [14]:
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:*::foundation-model/{foundation_model}",
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        },
        {
            "Sid": "AmazonBedrockAgentBedrockGetInferenceProfile",
            "Effect": "Allow",
            "Action":  [
                "bedrock:GetInferenceProfile",
                "bedrock:ListInferenceProfiles",
                "bedrock:UseInferenceProfile"
            ],
            "Resource": [
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

# agent_bedrock_policy = iam_client.create_policy(
#     PolicyName=agent_bedrock_allow_policy_name,
#     PolicyDocument=bedrock_policy_json
# )
try:
    agent_bedrock_policy = iam_client.create_policy(
        PolicyName=agent_bedrock_allow_policy_name,
        PolicyDocument=bedrock_policy_json
    )
except:
    # Usar política existente
    agent_bedrock_policy = iam_client.get_policy(
        PolicyArn=f"arn:aws:iam::{account_id}:policy/{agent_bedrock_allow_policy_name}"
    )



In [16]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {
            "Service": "bedrock.amazonaws.com"
        },
        "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
try:
    agent_role = iam_client.create_role(
        RoleName=agent_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )
except:
    agent_role = iam_client.get_role(RoleName=agent_role_name)

time.sleep(10)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '49f6374c-262b-46ac-98e7-09b75bb50627',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 19 Nov 2025 20:49:03 GMT',
   'x-amzn-requestid': '49f6374c-262b-46ac-98e7-09b75bb50627',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

Once the needed IAM role is created, you can use the Bedrock Agent client to create a new agent. To do so, you use the create_agent API. It requires an agent name, foundation model ID and instructions. You can also provide an agent description. 

In the next cell, run the following code:

In [18]:
# response = bedrock_agent_client.create_agent(
#     agentName=agent_name,
#     agentResourceRoleArn=agent_role['Role']['Arn'],
#     description=agent_description,
#     idleSessionTTLInSeconds=1800,
#     foundationModel=inference_profile,
#     instruction=agent_instruction,
# )
# agent_id = response['agent']['agentId']
# agent_id, response


try:
    response = bedrock_agent_client.create_agent(
        agentName=agent_name,
        agentResourceRoleArn=agent_role['Role']['Arn'],
        description=agent_description,
        idleSessionTTLInSeconds=1800,
        foundationModel=inference_profile,
        instruction=agent_instruction,
    )
    agent_id = response['agent']['agentId']
    print("Agent created successfully")

except bedrock_agent_client.exceptions.ConflictException:
    # Obtener agente existente
    agents = bedrock_agent_client.list_agents()
    for agent in agents['agentSummaries']:
        if agent['agentName'] == agent_name:
            agent_id = agent['agentId']
            response = bedrock_agent_client.get_agent(agentId=agent_id)
            break
    print("Using existing agent")

agent_id, response


Using existing agent


('SCG9G94YYP',
 {'ResponseMetadata': {'RequestId': 'ff66af00-5ace-49c2-8115-e3f80f3a3ff8',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'date': 'Wed, 19 Nov 2025 20:51:27 GMT',
    'content-type': 'application/json',
    'content-length': '15314',
    'connection': 'keep-alive',
    'x-amzn-requestid': 'ff66af00-5ace-49c2-8115-e3f80f3a3ff8',
    'x-amz-apigw-id': 'UTwYbHFaIAMEfSg=',
    'x-amzn-trace-id': 'Root=1-691e2dcf-613390eb087faadd0383938c'},
   'RetryAttempts': 0},
  'agent': {'agentId': 'SCG9G94YYP',
   'agentName': 'hr-assistant-function-def',
   'agentArn': 'arn:aws:bedrock:us-east-1:613468529489:agent/SCG9G94YYP',
   'clientToken': 'b570016d-da39-43b0-b01a-7f3b0b69b902',
   'instruction': 'You are an HR agent, helping employees understand HR policies and manage vacation time',
   'agentStatus': 'NOT_PREPARED',
   'foundationModel': 'us.amazon.nova-micro-v1:0',
   'description': 'Agent for providing HR assistance to manage vacation time',
   'orchestrationType': 'DEFAULT',
 

✏️ Note that the agent created is not yet prepared. Later, you will prepare and use the agent.

🎉 The agent has now been created, but it doesn't have any tools yet. In the next step, you'll create an action group to associate the tool to the agent.

## Task 4: Create Agent Action Group
You will now create an agent action group that uses the Lambda function created earlier. The 
create_agent_action_group
 function provides this functionality. You will use DRAFT as the agent version since you haven't yet created an agent version or alias. To inform the agent about the action group capabilities, you provide an action group description.

In this example, you provide the Action Group functionality using a functionSchema. You can alternatively provide an APISchema.

To define the functions using a function schema, you need to provide the name, description and parameters for each function.

You will first create the agent function schema. This is used to define the name, description, and parameters for the different actions available to the agent. 

In the next cell, in the Jupyter notebook, copy and paste the following code. Then, run the code:

In [19]:
agent_functions = [
    {
        'name': 'get_available_vacations_days',
        'description': 'get the number of vacations available for a certain employee',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee to get the available vacations",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'reserve_vacation_time',
        'description': 'reserve vacation time for a specific employee - you need all parameters to reserve vacation time',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee for which time off will be reserved",
                "required": True,
                "type": "integer"
            },
            "start_date": {
                "description": "the start date for the vacation time",
                "required": True,
                "type": "string"
            },
            "end_date": {
                "description": "the end date for the vacation time",
                "required": True,
                "type": "string"
            }
        }
    },
]

In [ ]:

# # Now, we can configure and create an action group here:
# agent_action_group_response = bedrock_agent_client.create_agent_action_group(
#     agentId=agent_id,
#     agentVersion='DRAFT',
#     actionGroupExecutor={
#         'lambda': lambda_function['FunctionArn']
#     },
#     actionGroupName=agent_action_group_name,
#     functionSchema={
#         'functions': agent_functions
#     },
#     description=agent_action_group_description
# )

# agent_action_group_response

try:
    agent_action_group_response = bedrock_agent_client.create_agent_action_group(
        agentId=agent_id,
        agentVersion='DRAFT',
        actionGroupExecutor={'lambda': lambda_function['FunctionArn']},
        actionGroupName=agent_action_group_name,
        functionSchema={'functions': agent_functions},
        description=agent_action_group_description
    )
except:
    # Usar ID del action group existente (del mensaje de error)
    agent_action_group_response = bedrock_agent_client.get_agent_action_group(
        agentId=agent_id,
        agentVersion='DRAFT',
        actionGroupId="PUZZO77ZVT"
    )

agent_action_group_response




Using existing action group


{'ResponseMetadata': {'RequestId': 'ac18a2ba-5562-417a-92f2-6e99487b9874',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 19 Nov 2025 20:53:23 GMT',
   'content-type': 'application/json',
   'content-length': '1388',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ac18a2ba-5562-417a-92f2-6e99487b9874',
   'x-amz-apigw-id': 'UTwqjFBioAMEr9Q=',
   'x-amzn-trace-id': 'Root=1-691e2e43-6412abc83ea524bb47a08eb5'},
  'RetryAttempts': 0},
 'agentActionGroup': {'agentId': 'SCG9G94YYP',
  'agentVersion': 'DRAFT',
  'actionGroupId': 'PUZZO77ZVT',
  'actionGroupName': 'VacationsActionGroup',
  'clientToken': '02f8e07e-f300-417f-afad-88b58ac09f05',
  'description': 'Actions for getting the number of available vacations days for an employee and confirm new time off',
  'createdAt': datetime.datetime(2025, 11, 19, 20, 26, 55, 333307, tzinfo=tzutc()),
  'updatedAt': datetime.datetime(2025, 11, 19, 20, 26, 55, 333307, tzinfo=tzutc()),
  'actionGroupExecutor': {'lambda': 'arn:aws:lambda

At this point, the action group has been created, but the AWS Lambda function resource policy needs to be modified to allow the agent to invoke the function associated with the action group.

Modify the resource policy on the AWS Lambda function to allow the specific bedrock agent to invoke it. 

In the next cell, run the following code:

In [22]:
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

🎉 The action group has been created and the agent has the needed permissions to invoke the AWS Lambda function. In the next step, you will prepare the agent and create an agent alias.

## Task 5: Prepare the Agent and Create an Agent Alias
The prepare_agent API creates a DRAFT version of the agent that can be used for internal testing.

To prepare the agent, in the next cell, copy and paste the following code. Then, run the code:

In [23]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
response

{'ResponseMetadata': {'RequestId': 'e41f9ac3-7b81-4965-9845-0c05d7a7f13c',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Wed, 19 Nov 2025 21:06:50 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e41f9ac3-7b81-4965-9845-0c05d7a7f13c',
   'x-amz-apigw-id': 'UTyorHrhIAMEMoA=',
   'x-amzn-trace-id': 'Root=1-691e316a-1524bd355f62320135ded987'},
  'RetryAttempts': 0},
 'agentId': 'SCG9G94YYP',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 11, 19, 21, 6, 50, 732968, tzinfo=tzutc())}

In [ ]:
agent_id=response.get('agentId')

'SCG9G94YYP'

In [25]:

response = bedrock_agent_client.create_agent_alias(
    agentAliasName='test-alias-1',
    agentId=agent_id
)

response

{'ResponseMetadata': {'RequestId': '72216562-f5ec-4ac4-b0f2-bf151da140eb',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 20 Nov 2025 01:40:57 GMT',
   'content-type': 'application/json',
   'content-length': '382',
   'connection': 'keep-alive',
   'x-amzn-requestid': '72216562-f5ec-4ac4-b0f2-bf151da140eb',
   'x-amz-apigw-id': 'UUayeFRqoAMEhhA=',
   'x-amzn-trace-id': 'Root=1-691e71a9-1e60d77b277aa1f6033d8ec9'},
  'RetryAttempts': 0},
 'agentAlias': {'agentId': 'SCG9G94YYP',
  'agentAliasId': 'W024RW0HFE',
  'agentAliasName': 'test-alias-1',
  'agentAliasArn': 'arn:aws:bedrock:us-east-1:613468529489:agent-alias/SCG9G94YYP/W024RW0HFE',
  'routingConfiguration': [{}],
  'createdAt': datetime.datetime(2025, 11, 20, 1, 40, 57, 258006, tzinfo=tzutc()),
  'updatedAt': datetime.datetime(2025, 11, 20, 1, 40, 57, 258006, tzinfo=tzutc()),
  'agentAliasStatus': 'CREATING',
  'aliasInvocationState': 'ACCEPT_INVOCATIONS'}}

In [27]:
agent_alias_id=response.get('agentAlias').get('agentAliasId')
agent_alias_id

'W024RW0HFE'

Locate and take note of the agentAliasId in the response output, as you will need it in the next step. Ensure you are copying the agentAliasId and not the agentId. These are two different values, and if you mix them up, the exercise will not work.

In the next cell, run the following command: agent_alias_id = "<FILL_ME_IN>"

Replace <FILL_ME_IN> with the agent alias ID. Ensure both of the < > characters are removed and only the ID in double quotes is present. Then, run the code.

## Task 6: Invoke the Agent
Now that you've created the agent, let's use the bedrock-agent-runtime client and the invoke_agent API to invoke this agent and perform some tasks.

In the next cell, in the Jupyter notebook, copy and paste the following code. Then, run it:

In [29]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="How much vacation does the employee with employee_id set to 1 have available?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2025-11-19 20:56:10,188] p16132 {1904821498.py:16} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Thu, 20 Nov 2025 01:56:10 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '157d7e69-c5b4-11f0-b975-047c1673d9e0',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'f99d5dbb-bb4d-41af-a331-96421b9024b3'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'f99d5dbb-bb4d-41af-a331-96421b9024b3',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x00000251C1105D10>,
 'contentType': 'application/json',
 'sessionId': '157d7e69-c5b4-11f0-b975-047c1673d9e0'}


[2025-11-19 20:56:13,067] p16132 {1904821498.py:23} INFO - Final answer ->
The employee with employee_id 1 has 12 days of vacation available.


✏️ This will invoke the agent with the prompt: How much vacation does the employee with employee_id set to 1 have available? The agent will think through the prompt, and choose to invoke the tool to look up the employee vacation balance. This will invoke the AWS Lambda function which will read the data from the SQLite database. The agent will receive the response from the function and work it into its final response.

Now try a follow up prompt to reserve one day off for the employee. In the next cell, run the following code:

In [32]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="Great. please reserve one day of time off for the employee with employee_id set to 1 for 2026-12-31",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2025-11-19 21:01:05,124] p16132 {1004788338.py:10} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Thu, 20 Nov 2025 02:01:05 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '157d7e69-c5b4-11f0-b975-047c1673d9e0',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '5b388f56-5c6e-4c71-ae53-4e86a30674e4'},
                      'HTTPStatusCode': 200,
                      'RequestId': '5b388f56-5c6e-4c71-ae53-4e86a30674e4',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x00000251C0561BA0>,
 'contentType': 'application/json',
 'sessionId': '157d7e69-c5b4-11f0-b975-047c1673d9e0'}


[2025-11-19 21:01:06,941] p16132 {1004788338.py:17} INFO - Final answer ->
<user__askuser question="Could you please provide the start date for the vacation time for the employee with employee_id 1?">


In [33]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="How much vacation does the employee with employee_id set to 1 have available?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2025-11-19 21:01:58,695] p16132 {140247262.py:10} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Thu, 20 Nov 2025 02:01:59 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '157d7e69-c5b4-11f0-b975-047c1673d9e0',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '0400a5c6-46f9-4d4f-a0d8-1771cacdb81a'},
                      'HTTPStatusCode': 200,
                      'RequestId': '0400a5c6-46f9-4d4f-a0d8-1771cacdb81a',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x00000251C0D028D0>,
 'contentType': 'application/json',
 'sessionId': '157d7e69-c5b4-11f0-b975-047c1673d9e0'}


[2025-11-19 21:02:00,334] p16132 {140247262.py:17} INFO - Final answer ->
The employee with employee_id 1 has 12 days of vacation available.


🎉 The agent was able to successfully use the tools to interact with the sample employee database based on a natural language prompt.

## Task 7: Cleanup
The next steps demonstrate how to delete the agent and associated resources. 

To delete the agent, you need to:

❌ Update the action group to disable it

❌ Delete agent action group

❌ Delete agent

❌ Delete lambda function

❌ Delete the created IAM roles and policies

Step 1: Update and delete the action group. In the next cell, in the Jupyter notebook, run the following code.

In [34]:
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    functionSchema={
        'functions': agent_functions
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

Step 2: Delete the agent alias.

In [35]:
response = bedrock_agent_client.delete_agent_alias(
    agentAliasId=agent_alias_id,
    agentId=agent_id
)

Step 3: Delete the agent.

In [36]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

Step 4: Delete the Lambda function.

In [37]:
lambda_client.delete_function(
    FunctionName=lambda_function_name
)

{'ResponseMetadata': {'RequestId': '3dbd73f2-d881-4de3-bb47-833304ba4de0',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Thu, 20 Nov 2025 02:04:48 GMT',
   'content-type': 'application/json',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3dbd73f2-d881-4de3-bb47-833304ba4de0'},
  'RetryAttempts': 1}}

Step 5: Delete the IAM roles and policies.

In [38]:
for policy in [agent_bedrock_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_function_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [agent_role_name, lambda_function_role]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)

⚠️🚨⚠️ If at any point you run into errors, you can also delete the resources using the AWS Management Console.

Exercise Complete
You now know how to:

✅ Create specialized Bedrock agents tasks

✅ Create an AWS Lambda function to use as a tool

✅ Prepare and create an alias for an agent

✅ Invoke an agent